In [24]:


import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score


/kaggle/input/docspot/datasets_228_482_diabetes.csv


Introduction

1.)[Data Collection and Analysis](#1)

2.)[Separating the label](#2)

3.)[Data standardization](#3)

4.)[Train Test Split](#4)

5.)[Training the Model](#5)

6.)[ Model Evaluation](#6)

7.)[Predictive system](#7)

<a id ="1"></a>
# Data Collection and Analysis

The dataset basically contains the informations of patients who has diabetes and who doesn't.It contains the data of females.There won't be any data for males.Various parameters Number of pregnancy they have gone through, their blood glucose, level insulin etc. are some of the fetures dataset contains.

In [25]:
#Loading the dataset
dia_df = pd.read_csv("datasets_228_482_diabetes.csv")

In [26]:
dia_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


* As you can see we have several features and the results.Pregnancies, glucosei blood pressure,skin thickness(taken from the triceps and basically tells us whether fat is strored in that particular muscle or not), insulin level, BMI(Body Mass Index, calculating by dividing the weight by height square), DiabetesPedigreeFunction(Number which indicates some diabetic value), age and outcome.The outcome part is our label.
* One represents the patient is diabetic and the zero represents the patient is non-diabetic.
* So we need to develop a system that can classify this data to either one or zero.

In [27]:
dia_df.shape

(768, 9)

* As you can see we have 768 rows and 9 columns, that means the data is collect from 768 people.
* Now let's get some statistical measures of this data frame.

In [28]:
dia_df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


* Now let's see how many case of examples for diabetic and non-diabetic examples


In [29]:
dia_df["Outcome"].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

* We have 500 non-diabetic and 269 diabetic patients in our dataset.
* The label 0 represents non-diabetic.
* The label 1 represents diabetic.
* Now we can try to get the mean for all this label zero and one.

In [30]:
dia_df.groupby("Outcome").mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


* This will give us mean values for both of them.ü
* We can see that the diabetic people have more glucose level on their blood.
* This feature could be important for us because this is what our machine learning algorithm will see.
* You can also see the age mean, and the Insulin level.It's obvious.
* It's a good practice that we take the mean to see the bigger picture.

<a id ="2"></a>
# Separating the label
* Let's separate this data and label("Outcome").We want all this data except the Outcome label in one particular location and we want the Outcome label separately.

In [31]:
x = dia_df.drop(columns = "Outcome" , axis = 1)
y = dia_df["Outcome"]

In [32]:
print(x)


     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0.351   31  


In [33]:
print(y)

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64


<a id ="3"></a>
# Data standardization
* So why we're doing this, you can see that every feature is in a range.If there is a difference in the range of all this values, it will be difficult for our machine learning model to make prediction.What we will do is that we will standardize the data in a particular range that helps our machine learning to make better predictions.
* At importing libraries we did import a function called standard scaler , so we're going to use that.

In [34]:
scaler = StandardScaler()

In [35]:
# Now we need to fit this dtaa in this scaler.
scaler.fit(x)

StandardScaler()

In [36]:
# Now we need to transform this data, for this we'll create another variable
stand_data = scaler.transform(x)

* Basically what we are doing is we are fitting all these inconsistent data with our standart scaler function and based on that standardization we are transforming all the data to a common range.
* Instead of using this fit and transform seperately you can do both in one line

In [37]:
print(stand_data)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


* As you can see all the values are in the range.

In [38]:
# Now we can make this simple by giving this standardized data to x again
x = stand_data

In [39]:
print(x)
print(y)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]
0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64


<a id ="4"></a>
# Train Test Split

In [40]:
x_train, x_test , y_train, y_test = train_test_split(x,y,test_size = 0.2,stratify = y,random_state = 31)

In [41]:
# Let's check the shapes
print(x.shape, x_train.shape, x_test.shape)

(768, 8) (614, 8) (154, 8)


<a id ="5"></a>
# Training the Model


In [42]:
classifier = svm.SVC(kernel = "linear")

In [43]:
#training the support vector machine classifier
classifier.fit(x_train,y_train)

SVC(kernel='linear')

<a id ="6"></a>
# Model Evaluation

First we will find the accuracy score on training data

In [44]:
# accuracy score on the training data
x_train_pred = classifier.predict(x_train)
x_train_accurcy = accuracy_score(x_train_pred, y_train)
print("Accuracy score of the train data : " , x_train_accurcy)

Accuracy score of the train data :  0.7768729641693811


Now we need to find the accuracy score on test data.

In [45]:
# accuracy score on the test data
x_test_pred = classifier.predict(x_test)
x_test_accurcy = accuracy_score(x_test_pred, y_test)
print("Accuracy score of the test data : " , x_test_accurcy)

Accuracy score of the test data :  0.7727272727272727


<a id ="7"></a>
# Predictive system
Once we give the data our machine learning model will tell whether the person is diabetic or non-diabetic.


In [46]:
input_data = (5,166,72,19,175,25.8,0.587,51)

#We have to change this input data into numpy array
input_data_numpy = np.asarray(input_data)

#we need to reshape this data
input_data_reshaped = input_data_numpy.reshape(1,-1)

#Since we standardize our training data before model, we need to standardize this data also
std_data = scaler.transform(input_data_reshaped)
print(std_data)

prediction = classifier.predict(std_data)
print(prediction)

if (prediction[0] == 0):
    print("The person is non-diabetic")
else:
    print("The person is diabetic")

[[ 0.3429808   1.41167241  0.14964075 -0.09637905  0.82661621 -0.78595734
   0.34768723  1.51108316]]
[1]
The person is diabetic


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


The idea behind the reshaping is that we feed our model with 768 data and it's expecting 768 data, we tell our model that we are giving only one data and wait for a predict.